In [1]:
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from tqdm import tqdm
from typing import Iterable, Callable
from questionanswering import squad

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
df = squad.parse_json_file("input/dev-v2.0.json")
df.drop_duplicates(["id"], inplace=True, ignore_index=True)
questions = list(df["question"])
contexts = list(df["context"])
df.info()

100%|██████████████████████████████████| 35/35 [00:00<00:00, 492.97it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11873 entries, 0 to 11872
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            11873 non-null  object
 1   title         11873 non-null  object
 2   question      11873 non-null  object
 3   answer_text   11873 non-null  object
 4   context       11873 non-null  object
 5   answer_start  11873 non-null  int16 
dtypes: int16(1), object(5)
memory usage: 487.1+ KB


In [4]:
assert len(df) == 11873
df.head()

,id,title,question,answer_text,context,answer_start
0,56ddde6b9a695914005b9628,Normans,In what country is Normandy located?,France,"The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (""Norman"" comes from ""Norseman"") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.",159
1,56ddde6b9a695914005b9629,Normans,When were the Normans in Normandy?,10th and 11th centuries,"The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (""Norman"" comes from ""Norseman"") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.",94
2,56ddde6b9a695914005b962a,Normans,From which countries did the Norse originate?,"Denmark, Iceland and Norway","The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (""Norman"" comes from ""Norseman"") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.",256
3,56ddde6b9a695914005b962b,Normans,Who was the Norse leader?,Rollo,"The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (""Norman"" comes from ""Norseman"") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.",308
4,56ddde6b9a695914005b962c,Normans,What century did the Normans first gain their separate identity?,10th century,"The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (""Norman"" comes from ""Norseman"") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish pop

In [5]:
s = pd.Series(np.where(df["answer_start"]==-1, True, False))
assert s.value_counts()[True] == 5945
s.value_counts(normalize=True)

True     0.500716
False    0.499284
dtype: float64

In [6]:
df[df["answer_start"] == -1].sample(5).head(5)

,id,title,question,answer_text,context,answer_start
10473,5ad031ef77cf76001a686df2,Imperialism,When did France take control of London?,,"France took control of Algeria in 1830 but began in earnest to rebuild its worldwide empire after 1850, concentrating chiefly in North and West Africa, as well as South-East Asia, with other conquests in Central and East Africa, as well as the South Pacific. Republicans, at first hostile to empire, only became supportive when Germany started to build her own colonial empire. As it developed, the new empire took on roles of trade with France, supplying raw materials and purchasing manufactured items, as well as lending prestige to the motherland and spreading French civilization and language as well as Catholicism. It also provided crucial manpower in both World Wars.",-1
478,5ad5648b5b96ef001a10ad97,Computational_complexity_theory,What does one not need to remain constant in a multiplication algorithm to produce the same outcome whether multiplying or squaring two integers?,,"The most commonly used reduction is a polynomial-time reduction. This means that the reduction process takes polynomial time. For example, the problem of squaring an integer can be reduced to the problem of multiplying two integers. This means an algorithm for multiplying two integers can be used to square an integer. Indeed, this can be done by giving the same input to both inputs of the multiplication algorithm. Thus we see that squaring is not more difficult than multiplication, since squaring can be reduced to multiplication.",-1
10096,5acfec5577cf76001a686500,Islamism,Who wrote a pamphlet outlining the non-radical ideas of the extremists?,,"The path of violence and military struggle was then taken up by the Egyptian Islamic Jihad organization responsible for the assassination of Anwar Sadat in 1981. Unlike earlier anti-colonial movements the extremist group directed its attacks against what it believed were ""apostate"" leaders of Muslim states, leaders who held secular leanings or who had introduced or promoted Western/foreign ideas and practices into Islamic societies. Its views were outlined in a pamphlet written by Muhammad Abd al-Salaam Farag, in which he states:",-1
2230,5ad419f8604f3c001a400499,Steam_engine,What is a main advantage of the efficiency cycle?,,"One of the principal advantages the Rankine cycle holds over others is that during the compression stage relatively little work is required to drive the pump, the working fluid being in its liquid phase at this point. By condensing the fluid, the work required by the pump consumes only 1% to 3% of the turbine power and contributes to a much higher efficiency for a real cycle. The benefit of this is lost somewhat due to the lower heat addition temperature. Gas turbines, for instance, have turbine entry temperatures approaching 1500 °C. Nonetheless, the efficiencies of actual large steam cycles and large modern gas turbines are fairly well matched.[citation needed]",-1
1970,5ad3f03c604f3c001a3ff829,Steam_engine,What is a term for the reversing of steam flow in a compression engine after each stroke?,,"In most reciprocating piston engines, the steam reverses its direction of flow at each stroke (counterflow), entering and exhausting from the cylinder by the same port. The complete engine cycle occupies one rotation of the crank and two piston strokes; the cycle also comprises four events – admission, expansion, exhaust, compression. These events are controlled by valves often working inside a steam chest adjacent to the cylinder; the valves distribute the steam by opening and closing steam ports communicating with the cylinder end(s) and are driven by valve gear, of which there are many types.[citation needed]",-1


In [7]:
%%time
col = "qc_length"
df[col] = df["question"].str.len() + df["context"].str.len() 
df[col] = df[col].astype(np.int32)

Wall time: 10 ms


In [8]:
%%time
col = "a_length"
df[col] = df["answer_text"].str.len()
df[col] = df[col].astype(np.int32)

Wall time: 7 ms


In [9]:
def word_length(cols: Iterable) -> Callable:
    def f(row) -> int:
        res = 0
        for col in cols:
            res += len(row[col].split())
        return res
    
    return f

In [10]:
%%time
col = "qc_word_length"
df[col] = df.progress_apply(word_length(["question", "context"]), axis=1)
df[col] = df[col].astype(np.int32)

100%|██████████████████████████| 11873/11873 [00:00<00:00, 48860.41it/s]

Wall time: 246 ms


In [11]:
%%time
col = "a_word_length"
df[col] = df.progress_apply(word_length(["answer_text"]), axis=1)
df[col] = df[col].astype(np.int32)

100%|█████████████████████████| 11873/11873 [00:00<00:00, 101479.58it/s]

Wall time: 120 ms


In [12]:
pretrained_dir = "../pretrained/google/electra-small-discriminator"
tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
print(f"{repr(tokenizer)}\n{tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='../pretrained/google/electra-small-discriminator', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
['input_ids', 'token_type_ids', 'attention_mask']


In [13]:
pretrained_dir = "../pretrained/albert-base-v2"
sp_tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
print(f"{repr(sp_tokenizer)}\n{sp_tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='../pretrained/albert-base-v2', vocab_size=30000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'cls_token': '[CLS]', 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False)})
['input_ids', 'token_type_ids', 'attention_mask']


In [14]:
pretrained_dir = "../pretrained/distilroberta-base"
bpe_tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
print(f"{repr(bpe_tokenizer)}\n{bpe_tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='../pretrained/distilroberta-base', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})
['input_ids', 'attention_mask']


In [15]:
%%time
x = tokenizer(questions, contexts)
print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
col = "qc_wp_length"
df[col] = [len(v) for v in x["input_ids"]]
df[col] = df[col].astype(np.int16)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
len=11873
Wall time: 5.63 s


In [16]:
%%time
x = sp_tokenizer(questions, contexts)
print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
col = "qc_sp_length"
df[col] = [len(v) for v in x["input_ids"]]
df[col] = df[col].astype(np.int16)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
len=11873
Wall time: 9.01 s


In [17]:
%%time
x = bpe_tokenizer(questions, contexts)
print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
col = "qc_bpe_length"
df[col] = [len(v) for v in x["input_ids"]]
df[col] = df[col].astype(np.int16)

dict_keys(['input_ids', 'attention_mask'])
len=11873
Wall time: 6.23 s


In [18]:
cols = ["qc_length", "a_length", "qc_word_length", "a_word_length", 
        "qc_wp_length", "qc_sp_length", "qc_bpe_length"]
df[cols].describe(percentiles=percentiles)

,qc_length,a_length,qc_word_length,a_word_length,qc_wp_length,qc_sp_length,qc_bpe_length
count,11873.000000,11873.000000,11873.000000,11873.000000,11873.000000,11873.000000,11873.000000
mean,870.387939,10.443022,137.926809,1.598922,180.744631,184.275836,181.706308
std,360.079804,18.382155,57.686171,2.845115,75.272238,77.352040,74.858027
min,208.000000,0.000000,33.000000,0.000000,41.000000,41.000000,42.000000
1%,345.000000,0.000000,57.000000,0.000000,74.000000,74.000000,75.000000
5%,556.000000,0.000000,85.000000,0.000000,108.000000,110.000000,110.000000
10%,584.000000,0.000000,91.000000,0.000000,118.000000,120.000000,119.000000
20%,626.000000,0.000000,99.000000,0.000000,130.000000,132.000000,131.000000
30%,667.000000,0.000000,106.000000,0.000000,139.000000,142.000000,140.000000
40%,716.000000,0.000000,114.000000,0.000000,150.000000,152.000000,150.000000


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11873 entries, 0 to 11872
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              11873 non-null  object
 1   title           11873 non-null  object
 2   question        11873 non-null  object
 3   answer_text     11873 non-null  object
 4   context         11873 non-null  object
 5   answer_start    11873 non-null  int16 
 6   qc_length       11873 non-null  int32 
 7   a_length        11873 non-null  int32 
 8   qc_word_length  11873 non-null  int32 
 9   a_word_length   11873 non-null  int32 
 10  qc_wp_length    11873 non-null  int16 
 11  qc_sp_length    11873 non-null  int16 
 12  qc_bpe_length   11873 non-null  int16 
dtypes: int16(4), int32(4), object(5)
memory usage: 742.2+ KB


In [20]:
%%time
df.to_parquet("output/dev.parquet", index=False)

Wall time: 54 ms
